# Motion correction using CaImAn and pixel correction


This Jupyter notebook performs batch motion correction on microscopy images using CaImAn and pixel value correction. It consists of three main sections:

Generate list of files:
Loads experiment metadata from Excel files specified in fileHeader
Creates lists of input files that need motion correction (files with "jumpCorrected.tif" but no "mc.tif" version)
Stores file paths and fps (frames per second) information
Motion correction:
Uses CaImAn's MotionCorrect to perform non-rigid motion correction on each file


Pixel value correction:
Corrects pixel intensity values that were altered during motion correction
For each motion corrected file:
Reads both original and motion corrected images
Computes average pixel value difference in a ROI (20-80% of image)
If difference >= 10:
Adjusts motion corrected image by subtracting/adding the difference
Saves corrected file
If difference < 10:
Assumes file already corrected


CaImAn for motion correction
tifffile for reading/writing TIFF files
pandas for handling metadata
numpy for array operations
The code is designed to process microscopy data from various experiments involving calcium imaging of inner hair cells (IHCs), spiral ganglion neurons (SGNs), and calcium waves.

In [ ]:
%pylab
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import os
import sys
sys.path.append('../src')

import seaborn as sns
import os

parameterFolder = '../parameters/'

# Select appropriate set of experiments, uncomment as required.
fileHeader = [
#'6N',
#'Atoh1_IHCs',
'Myo15_IHCs',
#'Myo15_IHCs_post-hearing',
#'NeuroD_SGN',
#'NeuroD_SGN_ex_vivo'
#'Pax2_Calciumwaves',
#'Pax2_Calciumwaves_broken',
# 'Pax2_Calciumwaves_post-hearing',
#'Snap25_SGN',
#'CavKO',
#'Otof KO',
#'FVB',
  ]

alldata = pd.DataFrame()
for h in fileHeader:
    inputFilename = os.path.join('../../',h)+'.xlsx'

    alldata1 = pd.read_excel(inputFilename)
    alldata = alldata.append(alldata1[alldata1['discard']!=1],ignore_index=True)

alldata = alldata[~alldata['Folder'].isna()]
alldata = alldata.reset_index()

# 1- Generate the list of files

Create a list of all the files that need to be motion corrected. This list contains the files that have a "jumpcorrected" file but not a motion corrected (mc) file. If a recording has a mc file in its folder is not going to be overwritten.

In [3]:
localDrive = 'E' # Select the drive where the data is saved in. 
localfolders = [localDrive+el[1:] for el in alldata['Folder']]
fpss = alldata['fps']
allfilesToCorrect = []
allinputFiles = []
allfps = []
for j,el in enumerate(localfolders):
  for (dirpath, dirnames, filenames) in os.walk(el):

    for name in filenames:
      if name =='1-jumpCorrected.tif':
        infile = os.path.join(dirpath, name)
        outFile = os.path.join(dirpath, '1-jumpCorrected-mc.tif')
        if not os.path.exists(outFile): 
          
          allfilesToCorrect.append(outFile) 
          allinputFiles.append(infile)
          allfps.append(fpss[j])

Define the function that does the motion correction (from CaImAn examples)

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

import tifffile

def caimanMC(inFile,outFile,fps):
    fnames = [inFile]
    # dataset dependent parameters
    fr = fps                            # imaging rate in frames per second
    decay_time = 1                  # length of a typical transient in seconds

    # motion correction parameters
    strides = (96, 96)          # start a new patch for pw-rigid motion correction every x pixels
    overlaps = (24*2, 24*2)         # overlap between pathes (size of patch strides+overlaps)
    max_shifts = (50,50)#(20,20)#(6,6)          # maximum allowed rigid shifts (in pixels)
    max_deviation_rigid = 25#10     # maximum shifts deviation allowed for patch with respect to rigid shifts
    pw_rigid = True             # flag for performing non-rigid motion correction

    # parameters for source extraction and deconvolution
    p = 1                       # order of the autoregressive system
    gnb = 2                     # number of global background components
    merge_thr = 0.85            # merging threshold, max correlation allowed
    rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
    stride_cnmf = 6             # amount of overlap between the patches in pixels
    K = 4                       # number of components per patch
    gSig = [4, 4]               # expected half size of neurons in pixels
    method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
    ssub = 1                    # spatial subsampling during initialization
    tsub = 1                    # temporal subsampling during intialization

    # parameters for component evaluation
    min_SNR = 2.0               # signal to noise ratio for accepting a component
    rval_thr = 0.85              # space correlation threshold for accepting a component
    cnn_thr = 0.99              # threshold for CNN based classifier
    cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected
    opts_dict = {'fnames': fnames,
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'gSig': gSig,
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest}

    opts = params.CNMFParams(params_dict=opts_dict)

    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)
    # first we create a motion correction object with the parameters specified
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    # note that the file is not loaded in memory
    mc.motion_correct(save_movie=True)
    m_els = cm.load(mc.fname_tot_els)
    border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0 
        # maximum shift to be used for trimming against NaNs
    m_els2 = m_els.copy()
    m_els2[m_els2<0]=0
    m_els2 = m_els2.astype(np.uint16)
    tifffile.imwrite(outFile, m_els2)
    cm.stop_server(dview=dview)
    

# 2- Actual motion correction. 
The mmap files generated by CaIman are deleted at the end, as they can be very big.

In [ ]:
for i,el in enumerate(allinputFiles):
    print(el)
    print(i)
    caimanMC(el,allfilesToCorrect[i],allfps[i])

    localfolders = [localDrive+el[1:] for el in alldata['Folder']]
    filestoRemove = []
    
    for el in localfolders:
        for (dirpath, dirnames, filenames) in os.walk(el):

            for name in filenames:
                if name.endswith('.mmap'):
                    print(dirpath)
                    os.remove(os.path.join(dirpath,name))

# 3- Correction of pixel values for motion corrected tiff files

CaIman alters the pixel value so that their distribution is different from the one of the original recording. We adjust the values by adding (or subtracting) the difference between the average pixel values in the mc recording compared to the original. Note that the jumpCorrected file must be present in the same folder of the mc file

In [7]:
import tifffile
import pandas as pd

In [ ]:
parameterFolder = '../../parameters/'
fileHeader = [
#'6N',
#'Atoh1_IHCs',
'Myo15_IHCs',
#'Myo15_IHCs_post-hearing',
#'NeuroD_SGN',
#'NeuroD_SGN_ex_vivo'
#'Pax2_Calciumwaves',
#'Pax2_Calciumwaves_broken',
# 'Pax2_Calciumwaves_post-hearing',
#'Snap25_SGN',
#'CavKO',
#'Otof KO',
#'FVB',
  ]

alldata = pd.DataFrame()
for h in fileHeader:
    inputFilename = os.path.join('../../',h)+'.xlsx'

    alldata1 = pd.read_excel(inputFilename)
    alldata = alldata.append(alldata1[alldata1['discard']!=1],ignore_index=True)

alldata = alldata[~alldata['Folder'].isna()]
alldata = alldata.reset_index()

In [9]:
localDrive = 'E' # Drive where the files are. Change Accordingly
localfolders = [localDrive+el[1:] for el in alldata['Folder']]
fpss = alldata['fps']
allfilesToCorrect = []
allinputFiles = []
allOrigFiles = []
allfps = []
for j,el in enumerate(localfolders):
  
    for (dirpath, dirnames, filenames) in os.walk(el):

      for name in filenames:
        if name =='1-jumpCorrected-mc.tif':
          infile = os.path.join(dirpath, name)

          outFile = os.path.join(dirpath, '1-jumpCorrected-mc.tif')
          orig = os.path.join(dirpath, '1-jumpCorrected.tif')
          if os.path.exists(infile):
            allOrigFiles.append(orig)
            allfilesToCorrect.append(outFile) 
            allinputFiles.append(infile)
            allfps.append(fpss[j])

In [ ]:
allinputFiles

In [ ]:
allfilesToCorrect

In [ ]:
allOrigFiles

In [ ]:
for i,el in enumerate(allinputFiles):
    """
    This script processes a list of input TIFF files, performs motion correction, and saves the corrected files if necessary.
    For each file in `allinputFiles`:
    1. Reads the motion-corrected TIFF file.
    2. Attempts to read the corresponding original (non-motion-corrected) TIFF file.
    3. If the original file is found:
        - Defines a region of interest (ROI) within the image.
        - Computes the difference (correction) between the motion-corrected and original images within the ROI.
        - If the mean correction value is significant (>= 10), adjusts the motion-corrected image by the mean correction value and saves the corrected image.
        - If the mean correction value is not significant, assumes the file is already corrected and prints a message.

    Libraries:
    - `tifffile`: Used for reading and writing TIFF files.
    """
    print(el)
    print(i)
    mcOriginal = tifffile.imread(el)
    try:
        notMCOriginal = tifffile.imread(allOrigFiles[i])
    except FileNotFoundError:
        print('Not found on local drive, tring the server')
        notMCOriginal = None
        
    if notMCOriginal is not None:
        xl1 = int(mcOriginal.shape[1]*0.2)
        xl2 = int(mcOriginal.shape[1]*0.8)

        yl1 = int(mcOriginal.shape[2]*0.2)
        yl2 = int(mcOriginal.shape[2]*0.8)

        corr = mcOriginal[:100,xl1:xl2,yl1:yl2].astype(float64)-notMCOriginal[:100,xl1:xl2,yl1:yl2].astype(float64)
        print(corr.mean())
        if abs(corr.mean())>=10:
            mcOriginal = mcOriginal.astype(float64) - corr.mean()
            tifffile.imwrite(allfilesToCorrect[i],mcOriginal.astype(uint16))
            print(allfilesToCorrect[i])
            print('\n\n')
        else:
            print('File probably already corrected, double check')
            print('\n\n')